## Шевцова Софья
группа 20930

15 вариант

Iris versicolor, Iris virginica Длина лепестка

In [63]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import datasets, model_selection
%matplotlib inline


def get_prior(y): # априорное распределение вероятностей
    num_classes = tf.reduce_max(y) + 1
    probs = np.zeros((num_classes,))
    for item in y:
        probs[item] +=1
    probs = probs/len(y)
    return tfd.Categorical(probs = probs)


def get_class_conditionals(x, y): # условное распределение
    num_classes = tf.reduce_max(y).numpy() + 1
    num_samples = x.shape[0]
    num_features = x.shape[1]
    loc = np.zeros((num_classes, num_features))
    scale_diag = np.zeros((num_classes, num_features))
    nos_in_class = np.zeros((num_classes,))
    for i in range(num_samples):
        nos_in_class[y[i]]+=1  
    nos_in_class = np.expand_dims(nos_in_class, axis = 1)    
    for i in range(num_samples):
        for j in range(num_features):
            loc[y[i], j] += x[i,j]      
    loc = loc/nos_in_class
    for i in range(num_samples):
        for j in range(num_features):
            scale_diag[y[i], j] += (x[i,j]-loc[y[i],j])**2   
    scale_diag = np.sqrt(scale_diag/nos_in_class)
    return tfd.MultivariateNormalDiag(loc = loc, scale_diag = scale_diag)


def predict_class(prior, class_conditionals, x): # вычисляет вероятности классов, возвращает тот, у котрого оан больше
    num_classes = tf.squeeze(class_conditionals.batch_shape)
    probs = []
    for i in range(num_classes):
        probs.append(tf.cast(class_conditionals[i].prob(x),dtype = tf.float32)*tf.cast(prior.probs[i], dtype = tf.float32))
    predictions = np.argmax(probs, axis = 0)
    return predictions


iris = datasets.load_iris() # загружаем базу ирисов
data = iris.data[:, 2:] # длины лепестков
data = data[:, :1]
targets = iris.target # названия ирисов
x_train, x_test, y_train, y_test = model_selection.train_test_split(data, targets, test_size=0.2) # делим на 80% тренировки и 20% тест
prior = get_prior(y_train)
class_conditionals = get_class_conditionals(x_train, y_train)
predictions = predict_class(prior, class_conditionals, x_test)
accuracy = accuracy_score(y_test, predictions) # точность предсказаний
print("Setosa = [0], versicolor = [1], virginica = [2]")
print("Для 1.0", predict_class(prior, class_conditionals, [[1.0]] ))
print("Для 1.9",predict_class(prior, class_conditionals, [[1.9]] ))
print("Для 2.45",predict_class(prior, class_conditionals, [[2.45]] ))
print("Для 3.0",predict_class(prior, class_conditionals, [[3.0]] ))
print("Для 4.5",predict_class(prior, class_conditionals, [[4.5]] ))
print("Для 4.8",predict_class(prior, class_conditionals, [[4.8]] ))
print("Для 5.1",predict_class(prior, class_conditionals, [[5.1]] ))
print("Для 6.0",predict_class(prior, class_conditionals, [[6.0]] ))



Setosa = [0], versicolor = [1], virginica = [2]
Для 1.0 [0]
Для 1.9 [0]
Для 2.45 [1]
Для 3.0 [1]


C:\Users\sofya\anaconda3\lib\site-packages\tensorflow_probability\python\distributions\internal\slicing.py:135: UserWarning: Unable to find property getter for parameter Tensor scale_diag on tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[3], event_shape=[1], dtype=float64), falling back to Distribution.dtype <dtype: 'float64'>
  warnings.warn('Unable to find property getter for parameter Tensor {} '


Для 4.5 [1]
Для 4.8 [1]
Для 5.1 [2]
Для 6.0 [2]


In [62]:
# 4.2
print('4.2. Оценить точность классификации при случайном разбиении выборки на обучающую (80%) и контрольную выборки (20%)')
print("Точность: {:.4f}".format(accuracy))

4.2. Оценить точность классификации при случайном разбиении выборки на обучающую (80%) и контрольную выборки (20%)
Точность: 0.9000
